[Source](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/SAM/Fine_tune_SAM_(segment_anything)_on_a_custom_dataset.ipynb)

In [3]:
import matplotlib.pyplot as plt
import os, pdb
import numpy as np
from PIL import Image
import pandas as pd

import torch.nn.functional as F
from typing import Tuple
from torch.nn import MSELoss
from transformers import SamModel 
from transformers import SamProcessor
from tqdm import tqdm
from statistics import mean
import torch
from torch.nn.functional import threshold, normalize
from torch.optim import Adam
import monai
from monai.transforms import ToTensor

from torchvision.transforms import ToPILImage, Compose, Resize, Grayscale, Lambda
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from datasets import load_dataset  

In [ ]:
DATA_DIR = '/sddata/data/MedSAM/public_test_data_01_10_2023'

In [ ]:
dataset = load_dataset("nielsr/breast-cancer", split="train")

In [ ]:

example = dataset[0]
image = example["image"]
type(image)

In [ ]:
label = example['label']
label = np.array(label) * 255
label

In [ ]:
label.shape

In [ ]:
np.array(label)[60:180,30:90]

In [ ]:
plt.imshow(label, cmap='gray')

In [ ]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
        # color = np.array([30/255, 144/255, 255/255, 0.6])
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    print(h, w)
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

fig, axes = plt.subplots()

axes.imshow(np.array(image))
ground_truth_seg = np.array(example["label"])
show_mask(ground_truth_seg, axes)
axes.title.set_text(f"Ground truth mask")
axes.axis("off")

## Create Pytorch Dataset

Below we define a regular PyTorch dataset, which gives us examples of the data prepared in the format for the model. Each example consists of:

* pixel values (which is the image prepared for the model)
* a prompt in the form of a bounding box
* a ground truth segmentation mask.
The function below defines how to get a bounding box prompt based on the ground truth segmentation. This was taken from here.

Note that SAM is always trained using certain "prompts", which you could be bounding boxes, points, text, or rudimentary masks. The model is then trained to output the appropriate mask given the image + prompt.

In [4]:
def get_bounding_box(ground_truth_map):
  # get bounding box from mask
  y_indices, x_indices = np.where(ground_truth_map > 0)
  x_min, x_max = np.min(x_indices), np.max(x_indices)
  y_min, y_max = np.min(y_indices), np.max(y_indices)
  # add perturbation to bounding box coordinates
  H, W = ground_truth_map.shape
  x_min = max(0, x_min - np.random.randint(0, 20))
  x_max = min(W, x_max + np.random.randint(0, 20))
  y_min = max(0, y_min - np.random.randint(0, 20))
  y_max = min(H, y_max + np.random.randint(0, 20))
  bbox = [x_min, y_min, x_max, y_max]

  return bbox

In [ ]:
class SAMDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = item["image"]
        ground_truth_mask = np.array(item["label"])

        # get bounding box prompt
        prompt = get_bounding_box(ground_truth_mask)

        # prepare image and prompt for the model
        inputs = self.processor(image, input_boxes=[[prompt]], return_tensors="pt")

        # remove batch dimension which the processor adds by default
        inputs = {k:v.squeeze(0) for k,v in inputs.items()}

        # add ground truth segmentation
        inputs["ground_truth_mask"] = ground_truth_mask

        return inputs

In [ ]:
processor = SamProcessor.from_pretrained("facebook/sam-vit-base")

In [ ]:
train_dataset = SAMDataset(dataset=dataset, processor=processor)

In [ ]:
example = train_dataset[0]
for k,v in example.items():
    print(k,v.shape)

## Create PyTorch DataLoader

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)

In [ ]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
    print(k,v.shape)

## Load the model


In [ ]:
model = SamModel.from_pretrained("facebook/sam-vit-base")

# make sure we only compute gradients for mask decoder
for name, param in model.named_parameters():
    if name.startswith("vision_encoder") or name.startswith("prompt_encoder"):
        param.requires_grad_(False)

In [ ]:
model

## Train the model

In [ ]:
# Note: Hyperparameter tuning could improve performance here
optimizer = Adam(model.mask_decoder.parameters(), lr=1e-5, weight_decay=0)

seg_loss = monai.losses.DiceCELoss(sigmoid=True, squared_pred=True, reduction='mean')

In [ ]:
num_epochs = 5

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

model.train()
epoch_losses = []
for epoch in range(num_epochs):
    batch_epoch_losses = []
    for batch in tqdm(train_dataloader):
        # forward pass
        outputs = model(pixel_values=batch["pixel_values"].to(device),
                        input_boxes=batch["input_boxes"].to(device),
                        multimask_output=False)
        
        # compute loss
        predicted_masks = outputs.pred_masks.squeeze(1)
        ground_truth_masks = batch["ground_truth_mask"].float().to(device)
        loss = seg_loss(predicted_masks, ground_truth_masks.unsqueeze(1))
        
        # backward pass (compute gradients of parameters w.r.t. loss)
        optimizer.zero_grad()
        loss.backward()
        
        # optimize
        optimizer.step()
        batch_epoch_losses.append(loss.item())

    print(f'EPOCH: {epoch}')
    print(f'Mean loss: {mean(batch_epoch_losses)}')
    epoch_losses.append(mean(batch_epoch_losses))
    plt.figure()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.plot(epoch_losses)
    plt.show()
    plt.close()

### Save it

In [ ]:
# Save the trained model
torch.save(model.state_dict(), "/sddata/data/MedSAM/model/facebook_sam-vit-base_BB.pth")

# Use our own data

In [ ]:
# Image
raw_image = Image.open('/sddata/data/MedSAM/public_test_data_01_10_2023/Color_Fundus/visivite_GA.png')
# Convert RGBA image to RGB
raw_image = raw_image.convert("RGB")

# Label
label = Image.open('/sddata/data/MedSAM/public_test_data_01_10_2023/labels/visivite_GA.png')

In [ ]:
raw_image

In [ ]:
label

In [ ]:
print(type(raw_image))
print(type(label))
print(np.array(raw_image).shape)
print(np.array(label).shape)

In [ ]:
print((np.array(label)[:,:,0] == 0).all()) # R
print((np.array(label)[:,:,1] == 0).all()) # G
print((np.array(label)[:,:,2] == 0).all()) # B
print((np.array(label)[:,:,3] == 0).all()) # A

In [ ]:
np.array(label)[800:900,800:900,0] # R

In [ ]:
w = np.array(label).shape[0]
h = np.array(label).shape[1]
ground_truth_seg = np.array(label)[:,:,0].reshape(w, h)
ground_truth_seg_norm = ground_truth_seg / 255
print(f"{w} {h}")

In [ ]:
fig, axes = plt.subplots()

axes.imshow(np.array(raw_image))
show_mask(ground_truth_seg_norm, axes, random_color=True)
axes.title.set_text(f"Ground truth mask")
axes.axis("off")

In [ ]:
class SAMDataset(Dataset):
    def __init__(self, dataset, processor, is_grayscale=False):
        if isinstance(dataset, str):
            self.dataset = pd.read_csv(os.path.join(dataset))
        else:
            self.dataset = dataset
        self.processor = processor
        self.is_grayscale = is_grayscale

    def __len__(self):
        return len(self.dataset)

    def _load_image(self, image_path, is_grayscale=True, resize=(256, 256)):
        image = Image.open(os.path.join(DATA_DIR,image_path))
        image = image.convert('RGB')
        # image = image.convert('RGB')
        img_transform = Compose([
            Resize(resize),
            Grayscale() if is_grayscale else Lambda(lambda x: x),
            # ToTensor() # it will convert to tensor and scale between 0-1
            Lambda(lambda x: torch.from_numpy(np.array(x))),
            
        ])
        image = img_transform(image)
        return image

    def __getitem__(self, idx):
        if isinstance(self.dataset, pd.DataFrame):
            item = self.dataset.iloc[idx]
            image = item['image_path_orig']
            label = item['labels']
            image = self._load_image(image, is_grayscale=False, resize=(256, 256))
            label = self._load_image(label, is_grayscale=True, resize=(256, 256))
            label = (label > 0).float()
        else:            
            item = self.dataset[idx]
            image = item["image"]
            label = np.array(item["label"])

        # get bounding box prompt
        # print(image)
        # print(type(image))
        # print(label.shape)
        prompt = get_bounding_box(label.squeeze())

        # prepare image and prompt for the model
        print(image.min(), image.max())
        inputs = self.processor(image, input_boxes=[[prompt]], return_tensors="pt")
        # print(image.min(), image.max())

        # remove batch dimension which the processor adds by default
        inputs = {k:v.squeeze(0) for k,v in inputs.items()}
        # inputs = {'image': image, 'label': label}

        # add ground truth segmentation
        inputs["ground_truth_mask"] = label

        return inputs

### Create Pytorch Dataset

In [ ]:
data_path = os.path.join(DATA_DIR, "image_key_cf.csv")
dataset_ours = SAMDataset(data_path, processor)

In [ ]:
index = 2
image = dataset_ours[index]['pixel_values']
label = dataset_ours[index]['ground_truth_mask']

In [ ]:
ToPILImage()(image)
ToPILImage()(label)

In [ ]:
train_dataset_ours_dataloader = DataLoader(dataset_ours, batch_size=3, shuffle=True)

In [ ]:
model = SamModel.from_pretrained("facebook/sam-vit-base")

# make sure we only compute gradients for mask decoder
for name, param in model.named_parameters():
    if name.startswith("vision_encoder") or name.startswith("prompt_encoder"):
        param.requires_grad_(False)

num_epochs = 15

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"
model.to(device)

model.train()
epoch_losses = []
for epoch in range(num_epochs):
    batch_epoch_losses = []
    for batch in tqdm(train_dataset_ours_dataloader):
        # forward pass
        outputs = model(pixel_values=batch["pixel_values"].to(device),
                        input_boxes=batch["input_boxes"].to(device),
                        multimask_output=False)
        
        # compute loss
        predicted_masks = outputs.pred_masks.squeeze(1)
        ground_truth_masks = batch["ground_truth_mask"].float().to(device)
        loss = seg_loss(predicted_masks, ground_truth_masks.unsqueeze(1))
        
        # backward pass (compute gradients of parameters w.r.t. loss)
        optimizer.zero_grad()
        loss.backward()
        
        # optimize
        optimizer.step()
        batch_epoch_losses.append(loss.item())

    print(f'EPOCH: {epoch}')
    print(f'Mean loss: {mean(batch_epoch_losses)}')
    epoch_losses.append(mean(batch_epoch_losses))
    plt.figure()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.plot(epoch_losses)
    plt.show()
    plt.close()

### Create Pytorch DataLoader

### Train The Model